# API Integration, Natural Language Processing, Data Cleaning, Calculations and Updating SQL Database for Machine Learning

Initial Imports

In [2]:
# Initial imports
import pandas as pd
from pathlib import Path
import requests
from dotenv import load_dotenv
import os
import numpy as np
import panel as pn
from datetime import date
import datetime
import json
import time
import simfin as sf

# Load data to get API Keys from an env file
load_dotenv()

True

Import Custom Functions from Python file

In [77]:
import custom_functions as cf

Setting up API and downloading data

In [4]:
# API key for SimFin

# # SimFin API
# simfin_api_key = os.getenv("SIMFIN_API_KEY")

# SimFin free API key
sf.set_api_key('free')

In [5]:
# Set the local directory where data-files are stored.
# The directory will be created if it does not already exist.
sf.set_data_dir('~/simfin_data/')

In [6]:
# Download the data from the SimFin server and load into a Pandas DataFrame

    # Cash Flow Download
# TTM
df_ttm_cash_flow = sf.load_cashflow(variant='ttm', market='us')
    # Income Statement Download
# TTM
df_ttm_income_st = sf.load_income(variant='ttm', market='us')
    # Balance Sheet Download
# TTM
df_ttm_balance_st = sf.load_balance(variant='ttm', market='us')

Dataset "us-cashflow-ttm" on disk (1 days old).
- Loading from disk ... Done!
Dataset "us-income-ttm" on disk (1 days old).
- Loading from disk ... Done!
Dataset "us-balance-ttm" on disk (1 days old).
- Loading from disk ... Done!


In [7]:
    # Share Prices Download
# Daily
df_d_shares = sf.load_shareprices(variant='daily', market='us')

Dataset "us-shareprices-daily" on disk (1 days old).
- Loading from disk ... Done!


In [8]:
    # Company's IDs Download
df_c_id = sf.load_companies(market='us')
    # Sector/Indistry Download
df_sect_industry = sf.load_industries()
    # Market IDs Download
df_m_id = sf.load_markets()

Dataset "us-companies" on disk (1 days old).
- Loading from disk ... Done!
Dataset "industries" on disk (1 days old).
- Loading from disk ... Done!
Dataset "markets" on disk (1 days old).
- Loading from disk ... Done!


Clean Data

In [9]:
# Daily DataFrame drop columns
price_data = df_d_shares.copy()
price_data.drop(columns = ['SimFinId', 'Open', 'Low', 'High', 'Shares Outstanding'], inplace=True)

# Second price dataframe to get around an error I was getting in the forwardfill later in the code with the merging of the prices and fundimental data
price_data2 = price_data.copy()
price_data2.drop(columns = ['Adj. Close', 'Dividend', 'Volume'], inplace=True)
price_data2.rename(columns = {'Close': 'Close Delete'}, inplace = True)

In [10]:
# TTM DataFrame drop columns
cash_flow_ttm_data = df_ttm_cash_flow.copy()
cash_flow_ttm_data.drop(columns = ['SimFinId', 'Currency', 'Restated Date'], inplace=True)

income_st_ttm_data = df_ttm_income_st.copy()
income_st_ttm_data.drop(columns = [
    'SimFinId', 'Currency', 'Fiscal Year', 'Fiscal Period', 'Restated Date', 'Shares (Basic)', 
    'Shares (Diluted)', 'Publish Date', 'Depreciation & Amortization'], inplace=True)

balance_st_ttm_data = df_ttm_balance_st.copy()
balance_st_ttm_data.drop(columns = [
    'SimFinId', 'Currency', 'Fiscal Year', 'Fiscal Period', 'Restated Date', 'Shares (Basic)', 
    'Shares (Diluted)', 'Publish Date'], inplace=True)

# TTM DataFrame Combine Fundimentals
ttm_fund_data_combined = pd.concat([cash_flow_ttm_data, income_st_ttm_data, balance_st_ttm_data], axis = 'columns', join = 'inner')

Piotroski F-score

In [11]:
# Calculate the Piotroski F-score before adding in price data
ttm_fund_data_combined['Piotroski F-score'] = cf.Piotroski_F_score(
    ttm_fund_data_combined,
    'Net Income (Common)',
    'Total Assets',
    'Net Cash from Operating Activities',
    'Total Noncurrent Liabilities',
    'Total Current Assets',
    'Total Current Liabilities',
    'Shares (Diluted)',
    'Gross Profit',
    'Revenue')

Cleaning data continued and merging data (Remove financials and real estate stocks as they are completely different and standard practice for backtesting)

In [12]:
# Merged in Sector and Industry
ttm_fund_data_combined = ttm_fund_data_combined.merge(df_c_id, left_index = True, right_index = True)
ttm_fund_data_combined = ttm_fund_data_combined.join(df_sect_industry, on = 'IndustryId')
ttm_fund_data_combined = ttm_fund_data_combined.reset_index().set_index(['Ticker', 'Publish Date'])

# Remove financials and real estate stocks
ttm_fund_data_combined = ttm_fund_data_combined[ttm_fund_data_combined['Sector'] != 'Financial Services']
ttm_fund_data_combined = ttm_fund_data_combined[ttm_fund_data_combined['Sector'] != 'Real Estate']

# Revome tickers that are not in both price and fundimentals
fundl_tickers = ttm_fund_data_combined.index.unique(level = 0)
price_tickers = price_data.index.unique(level = 0)
l_func = lambda x, y: list((set(x)- set(y))) + list((set(y)- set(x)))
unmached = l_func(fundl_tickers, price_tickers)
clean_price_data = price_data.copy().drop(unmached, errors='ignore')
clean_price_data2 = price_data2.copy().drop(unmached, errors='ignore')
clean_ttm_fund_data_combined = ttm_fund_data_combined.copy().drop(unmached, errors='ignore')

In [13]:
# Merge in Price
ttm_data_combined_price = pd.concat([clean_price_data, clean_ttm_fund_data_combined], axis = 1)

In [14]:
# fill fundimental data for each day of price data to calculate ratios each day
ttm_data_combined_price_fill = ttm_data_combined_price.copy().fillna(method = 'ffill')

# Drop NA
ttm_data_all = ttm_data_combined_price_fill.copy().dropna()

In [15]:
# TTM DataFrame Combine Fundimentals & Price to get rid of all the bad pricing
ttm_data_all = pd.concat([clean_price_data2, ttm_data_all], axis = 1)
ttm_data_all = ttm_data_all.dropna()

# Drop all the extra columns that are useless
ttm_data_all.drop(columns = ['Close Delete', 'Dividend','Net Income/Starting Line', 'Net Income', 'SimFinId'], inplace=True)

Select to trim data to make calculations less computationaly intensive and faster

In [16]:
# 0 - Empty Database (No Trim)
# 1 - Update Database (Trim)
create_update_choice = 0
if create_update_choice == 1:
    ttm_data_all = ttm_data_all.reset_index().set_index(['level_0'])
    ttm_data_all = ttm_data_all.loc['2018-12-31':]
    ttm_data_all = ttm_data_all.reset_index().set_index(['level_0', 'level_1'])
    ttm_data_all = ttm_data_all.sort_index()
else:
    pass

In [17]:
# Add Market Cap
ttm_data_all['Market Cap'] = ttm_data_all['Close'] * ttm_data_all['Shares (Diluted)']

In [18]:
# dropping some columns now to reduce the size of the dataframe
column_single =  [
    'Report Date', 'Fiscal Year', 'Fiscal Period', 'Shares (Basic)', 'Shares (Diluted)', 'Company Name', 'IndustryId',
    'Non-Cash Items', 'Change in Working Capital', 'Change in Inventories', 'Change in Accounts Payable',
    'Net Change in Long Term Investment', 'Cost of Revenue', 'Operating Expenses', 'Selling, General & Administrative',
    'Non-Operating Income (Loss)', 'Abnormal Gains (Losses)', 'Net Extraordinary Gains (Losses)', 'Payables & Accruals',
    'Share Capital & Additional Paid-In Capital', 'Treasury Stock', 'Retained Earnings'
    ]
columns = column_single
ttm_data_all.drop(columns = columns, inplace=True)

In [20]:
# Add % Change

    # Creates a list of tickers for index values
ticker_list = ttm_data_all.copy().reset_index().set_index(['level_0']).index.drop_duplicates()

    # Create an empty dataframe that will be the combined version
combined_df = pd.DataFrame()
#single_change = ttm_data_all['Close'].copy()

count = 0
for ticker in ticker_list:
    single_change = ttm_data_all[['Close']].copy().loc[[ticker]]
    single_change['Close Chg'] = ( single_change['Close'] / single_change['Close'].shift(1) ) -1
    
    # add dataframes topgether
    combined_df = combined_df.append(single_change)
    
    # Calculates percentage of completion
    count = count + 1
    percent = (count/len(ticker_list)) * 100
    time.sleep(.01)
    print(f"{round(percent, 1)}%", end="\r")


# Add returns to main dataframe and drop N/A from percent calculation
ttm_data_all = pd.concat([ttm_data_all, combined_df['Close Chg']], axis = 1)
ttm_data_all = ttm_data_all.dropna()

100.0%


### ADD UNIVERSE, SECTOR AND INDUSTRY DATA SUMS FOR COMPARIBLES

In [3]:
# Specify the end of column name
# 'level_0 ' = ''
col_name_total = " Universe"
col_name_sector = " Sector"
col_name_industry = " Industry"

# list to rename calculations for Stock, Total, Sector and Industry data in a for loop
col_name_loop = ['', col_name_total, col_name_sector, col_name_industry]

In [22]:
# Create empty dictionaries to reduce merging time for Universe, Sector and Industry Sum
universe_sum_df = pd.concat([ttm_data_all['Sector'], ttm_data_all['Industry']], axis = 1)
sector_sum_df = pd.concat([ttm_data_all['Sector'], ttm_data_all['Industry']], axis = 1)
industry_sum_df = pd.concat([ttm_data_all['Sector'], ttm_data_all['Industry']], axis = 1)

In [23]:
# All data sum

# Clean columns to get sum and column names to have the same amount of column names for Total data
ttm_data_all_clean_all = ttm_data_all.copy().reset_index().set_index(['level_0', 'level_1'])
ttm_data_all_clean_all.drop(columns = [
    'Close', 'Adj. Close','Sector',
    'Industry', 'Piotroski F-score'], inplace=True)


# Creat Total Data to be added to main dataframe

# Sum all the data
ttm_data_all_clean_all = ttm_data_all_clean_all.copy().reset_index().groupby(['level_1']).sum()

# Use col_name_change funtion to change column names with specified name at end
ttm_data_all_clean_all.columns = cf.col_name_change(ttm_data_all_clean_all, col_name_total)

In [24]:
# Create weighted Average for the Universe

# Create data frame to merge with Universe before loop
ttm_data_all_clean_all_weighted = ttm_data_all.copy().reset_index().set_index(['level_1'])

# Take the market cap of the Universe and put it in the temparary Dataframe for weighted averages
ttm_data_all_clean_all_weighted[f'Market Cap{col_name_total}'] = ttm_data_all_clean_all[f'Market Cap{col_name_total}']

# Weighted Return
ttm_data_all_clean_all_weighted[f'Weighted Return{col_name_total}'] = cf.weighted_average(
    ttm_data_all_clean_all_weighted,
    'Close Chg',
    'Market Cap',
    col_name_total)

# Weighted Piotroski F-score
ttm_data_all_clean_all_weighted[f'Weighted Piotroski F-score{col_name_total}'] = cf.weighted_average(
    ttm_data_all_clean_all_weighted,
    'Piotroski F-score',
    'Market Cap',
    col_name_total)

# Sum up the 
ttm_data_all_clean_all_weighted = ttm_data_all_clean_all_weighted.groupby(['level_1']).sum()

# Merge in weighted data after sum
ttm_data_all_clean_all = pd.concat([ttm_data_all_clean_all, ttm_data_all_clean_all_weighted[f'Weighted Return{col_name_total}'], ttm_data_all_clean_all_weighted[f'Weighted Piotroski F-score{col_name_total}']], axis = 1)

In [25]:
# Sector Data

# Clean columns to get sum and column names to have the same amount of column names for Sector data
ttm_data_all_clean_sector = ttm_data_all.copy().reset_index().set_index(['level_0', 'level_1'])
ttm_data_all_clean_sector.drop(columns = [
    'Close', 'Adj. Close', 'Industry',
    'Piotroski F-score'], inplace=True)


# Creat Sector Data to be added to main dataframe

# Sum Sector data
ttm_data_all_clean_sector = ttm_data_all_clean_sector.copy().reset_index().groupby(['Sector', 'level_1']).sum()

# Use col_name_change funtion to change column names with specified name at end
ttm_data_all_clean_sector.columns = cf.col_name_change(ttm_data_all_clean_sector, col_name_sector)

In [26]:
# Create weighted Average for the Sector

# Create data frame to merge with Sector before loop
ttm_data_all_clean_sector_weighted = ttm_data_all.copy().reset_index().set_index(['Sector', 'level_1'])

# Take the market cap of the Sector and put it in the temparary Dataframe for weighted averages
ttm_data_all_clean_sector_weighted[f'Market Cap{col_name_sector}'] = ttm_data_all_clean_sector[f'Market Cap{col_name_sector}']

# Weighted Return
ttm_data_all_clean_sector_weighted[f'Weighted Return{col_name_sector}'] = cf.weighted_average(
    ttm_data_all_clean_sector_weighted,
    'Close Chg',
    'Market Cap',
    col_name_sector)

# Weighted Piotroski F-score
ttm_data_all_clean_sector_weighted[f'Weighted Piotroski F-score{col_name_sector}'] = cf.weighted_average(
    ttm_data_all_clean_sector_weighted,
    'Piotroski F-score',
    'Market Cap',
    col_name_sector)

# Sum up the 
ttm_data_all_clean_sector_weighted = ttm_data_all_clean_sector_weighted.groupby(['Sector', 'level_1']).sum()

# Merge in weighted data after sum
ttm_data_all_clean_sector = pd.concat([ttm_data_all_clean_sector, ttm_data_all_clean_sector_weighted[f'Weighted Return{col_name_sector}'], ttm_data_all_clean_sector_weighted[f'Weighted Piotroski F-score{col_name_sector}']], axis = 1)

In [27]:
# Industry Data

# Clean columns to get sum and column names to have the same amount of column names for Industry data
ttm_data_all_clean_industry = ttm_data_all.copy().reset_index().set_index(['level_0', 'level_1'])
ttm_data_all_clean_industry.drop(columns = [
    'Close', 'Adj. Close','Sector',
    'Piotroski F-score'], inplace=True)


# Creat Industry Data to be added to main dataframe

# Sum Industry data
ttm_data_all_clean_industry = ttm_data_all_clean_industry.copy().reset_index().groupby(['Industry', 'level_1']).sum()

# Use col_name_change funtion to change column names with specified name at end
ttm_data_all_clean_industry.columns = cf.col_name_change(ttm_data_all_clean_industry, col_name_industry)

In [28]:
# Create weighted Average for the Industry

# Create data frame to merge with Sector before loop
ttm_data_all_clean_industry_weighted = ttm_data_all.copy().reset_index().set_index(['Industry', 'level_1'])

# Take the market cap of the Sector and put it in the temparary Dataframe for weighted averages
ttm_data_all_clean_industry_weighted[f'Market Cap{col_name_industry}'] = ttm_data_all_clean_industry[f'Market Cap{col_name_industry}']

# Weighted Return
ttm_data_all_clean_industry_weighted[f'Weighted Return{col_name_industry}'] = cf.weighted_average(
    ttm_data_all_clean_industry_weighted,
    'Close Chg',
    'Market Cap',
    col_name_industry)

# Weighted Piotroski F-score
ttm_data_all_clean_industry_weighted[f'Weighted Piotroski F-score{col_name_industry}'] = cf.weighted_average(
    ttm_data_all_clean_industry_weighted,
    'Piotroski F-score',
    'Market Cap',
    col_name_industry)

# Sum up the 
ttm_data_all_clean_industry_weighted = ttm_data_all_clean_industry_weighted.groupby(['Industry', 'level_1']).sum()

# Merge in weighted data after sum
ttm_data_all_clean_industry = pd.concat([ttm_data_all_clean_industry, ttm_data_all_clean_industry_weighted[f'Weighted Return{col_name_industry}'], ttm_data_all_clean_industry_weighted[f'Weighted Piotroski F-score{col_name_industry}']], axis = 1)

In [29]:
# Join Universe compareables to stock data
join_stock = 'level_0'
universe_sum_df = cf.join_obj_loop(universe_sum_df, ttm_data_all_clean_all, join_stock)

# Add other daily API data here or fill monthly/qur/yearly data here to universe_sum_df
# spy_alpaca_data

In [30]:
# Join Sector compareables to stock data
join_sector = 'Sector'
sector_sum_df = cf.join_obj_loop(sector_sum_df, ttm_data_all_clean_sector, join_sector)

In [31]:
# Join Industry compareables to stock data
join_industry = 'Industry'
industry_sum_df = cf.join_obj_loop(industry_sum_df, ttm_data_all_clean_industry, join_industry)

In [32]:
# Reset indexes of comparable data to get ready to merge
universe_sum_df = universe_sum_df.reset_index().set_index(['level_0', 'level_1'])
sector_sum_df = sector_sum_df.reset_index().set_index(['level_0', 'level_1'])
industry_sum_df = industry_sum_df.reset_index().set_index(['level_0', 'level_1'])

In [33]:
comparable_data_all = pd.concat([ttm_data_all, universe_sum_df, sector_sum_df, industry_sum_df], axis = 1)

### Checkpoint

In [34]:
# save as pickle to make a checkpoint
comparable_data_all.to_pickle("Resources/pickle_file_comparable_data_all.pkl")

In [4]:
comparable_data_all = pd.read_pickle("Resources/pickle_file_comparable_data_all.pkl")

## Calculate Ratios for Stocks, Universe, Sector and Industry

In [5]:
# Copy dataframe to make calculations
comparable_calculations = comparable_data_all.copy()

In [6]:
# CALCULATIONS (Might need to break up in different cells to run)

count = 0
for i in col_name_loop:
    
# Enterprise Value

    # Enterprise Value (Does not include long term cash)

    comparable_calculations[f"Enterprise Value{i}"] = cf.enterprise_value(
        comparable_calculations,
        f'Market Cap{i}',
        f'Short Term Debt{i}',
        f'Long Term Debt{i}',
        f'Cash, Cash Equivalents & Short Term Investments{i}')

# Shareholder Value

    # Total Shareholder Value
    comparable_calculations[f"Total Shareholder Value{i}"] = cf.shareholder_value(
        comparable_calculations,
        f'Cash from (Repurchase of) Equity{i}',
        f'Cash from (Repayment of) Debt{i}',
        f'Dividends Paid{i}')

    # Shareholder Yield
    comparable_calculations[f"Total Shareholder Yield{i}"] = cf.shareholder_yield(
        comparable_calculations,
        f'Total Shareholder Value{i}',
        f'Market Cap{i}')

    # Shareholder EV Yield
    comparable_calculations[f"Total Shareholder EV Yield{i}"] = cf.shareholder_yield(
        comparable_calculations,
        f'Total Shareholder Value{i}',
        f"Enterprise Value{i}")

    # Div + Repurchase Value
    comparable_calculations[f"Div + Repurchase Value{i}"] = cf.div_repurchase_value(
        comparable_calculations,
        f'Cash from (Repurchase of) Equity{i}',
        f'Dividends Paid{i}')

    # Div + Repurchase Yield
    comparable_calculations[f"Div + Repurchase Yield{i}"] = cf.div_repurchase_yield(
        comparable_calculations,
        f'Div + Repurchase Value{i}',
        f"Market Cap{i}")

    # Div + Repurchase EV Yield
    comparable_calculations[f"Div + Repurchase EV Yield{i}"] = cf.div_repurchase_yield(
        comparable_calculations,
        f'Div + Repurchase Value{i}',
        f"Enterprise Value{i}")

# P/E Ratio

    # P/E (Market Cap to Earnings)
    comparable_calculations[f"PE Market Cap to Earnings{i}"] = cf.pe_ratio(
        comparable_calculations,
        f'Market Cap{i}',
        f'Net Income (Common){i}')

    # EV/E (EV to Earnings)
    comparable_calculations[f"PE Market Cap to Earnings{i}"] = cf.pe_ratio(
        comparable_calculations,
        f"Enterprise Value{i}",
        f'Net Income (Common){i}')
    
# Coverage Ratios

    # Earnings Coverage Ratio
    comparable_calculations[f"Earnings Coverage Ratio{i}"] = cf.coverage_ratio(
        comparable_calculations,
        f"Net Income (Common){i}",
        f'Dividends Paid{i}')
    
# Other Yield Ratios

    # EBIT Value
    comparable_calculations[f"EBIT{i}"] = cf.ebit(
        comparable_calculations,
        f'Net Income (Common){i}',
        f'Interest Expense, Net{i}',
        f'Income Tax (Expense) Benefit, Net{i}')

    # EBIT Yield
    comparable_calculations[f"EBIT Yield{i}"] = cf.yield_calc(
        comparable_calculations,
        f'EBIT{i}',
        f'Market Cap{i}')

    # EBIT EV Yield
    comparable_calculations[f"EBIT EV Yield{i}"] = cf.yield_calc(
        comparable_calculations,
        f'EBIT{i}',
        f"Enterprise Value{i}")

    # EBITDA Value
    comparable_calculations[f"EBITDA{i}"] = cf.ebitda(
        comparable_calculations,
        f'EBIT{i}',
        f'Depreciation & Amortization{i}',)

    # EBITDA Yield
    comparable_calculations[f"EBITDA Yield{i}"] = cf.yield_calc(
        comparable_calculations,
        f'EBITDA{i}',
        f'Market Cap{i}')
        
    # EBITDA EV Yield
    comparable_calculations[f"EBITDA EV Yield{i}"] = cf.yield_calc(
        comparable_calculations,
        f'EBITDA{i}',
        f"Enterprise Value{i}")

    # Revenue Yield
    comparable_calculations[f"Revenue Yield{i}"] = cf.yield_calc(
        comparable_calculations,
        f'Revenue{i}',
        f'Market Cap{i}')

    # Revenue EV Yield
    comparable_calculations[f"Revenue EV Yield{i}"] = cf.yield_calc(
        comparable_calculations,
        f'Revenue{i}',
        f"Enterprise Value{i}")

    # Operating Yield
    comparable_calculations[f"Operating Yield{i}"] = cf.yield_calc(
        comparable_calculations,
        f'Net Cash from Operating Activities{i}',
        f'Market Cap{i}')

    # Operating Ev Yield
    comparable_calculations[f"Operating EV Yield{i}"] = cf.yield_calc(
        comparable_calculations,
        f'Net Cash from Operating Activities{i}',
        f"Enterprise Value{i}")

    # Cash to Market Cap Yield (Does not include long term cash & equivilents)
    comparable_calculations[f"Cash Yield{i}"] = cf.yield_calc(
        comparable_calculations,
        f'Cash, Cash Equivalents & Short Term Investments{i}',
        f'Market Cap{i}')
    
    
    # Calculates percentage of completion
    count = count + 1
    percent = (count/len(col_name_loop)) * 100
    time.sleep(.01)
    print(f"{round(percent, 1)}%", end="\r")

### Clean Data for Model Use

In [7]:
# Drop useles features
clean_data = comparable_calculations.copy()

column_loop_sum = []

for i in col_name_loop:
    column_loop = [
        f'Depreciation & Amortization{i}', f'Change in Accounts Receivable{i}', f'Change in Other{i}',
        f'Net Cash from Operating Activities{i}', f'Change in Fixed Assets & Intangibles{i}',
        f'Net Cash from Acquisitions & Divestitures{i}', f'Net Cash from Investing Activities{i}',
        f'Dividends Paid{i}', f'Cash from (Repayment of) Debt{i}', f'Cash from (Repurchase of) Equity{i}',
        f'Net Cash from Financing Activities{i}',f'Net Change in Cash{i}', f'Revenue{i}', f'Gross Profit{i}',
        f'Research & Development{i}', f'Operating Income (Loss){i}', f'Interest Expense, Net{i}',
        f'Pretax Income (Loss), Adj.{i}', f'Pretax Income (Loss){i}', f'Income Tax (Expense) Benefit, Net{i}',
        f'Income (Loss) from Continuing Operations{i}',f'Net Income (Common){i}',
        f'Cash, Cash Equivalents & Short Term Investments{i}', f'Accounts & Notes Receivable{i}', f'Inventories{i}',
        f'Total Current Assets{i}', f'Property, Plant & Equipment, Net{i}', f'Long Term Investments & Receivables{i}',
        f'Other Long Term Assets{i}', f'Total Noncurrent Assets{i}', f'Total Assets{i}', f'Short Term Debt{i}',
        f'Total Current Liabilities{i}', f'Long Term Debt{i}', f'Total Noncurrent Liabilities{i}',
        f'Total Liabilities{i}', f'Total Equity{i}', f'Total Liabilities & Equity{i}', f'Enterprise Value{i}',
        f'Total Shareholder Value{i}', f'Div + Repurchase Value{i}', f'EBIT{i}', f'EBITDA{i}'
    ]
    column_loop_sum = column_loop_sum + column_loop

columns_single = ['Yield', 'Close Chg Universe', 'Close Chg Sector', 'Close Chg Industry', 'Shareholder Value']

columns = column_loop_sum + columns_single
clean_data.drop(columns = columns, inplace=True)

clean_data['Close Chg'] = clean_data['Close Chg'].round(6)
clean_data['Weighted Return Universe'] = clean_data['Weighted Return Universe'].round(6)
clean_data['Weighted Return Sector'] = clean_data['Weighted Return Sector'].round(6)
clean_data['Weighted Return Industry'] = clean_data['Weighted Return Industry'].round(6)

clean_data = clean_data.loc[:,~clean_data.columns.duplicated()]

In [73]:
clean_data = pd.read_pickle("Resources/clean_data.pkl")

In [74]:
clean_data_aapl = clean_data.copy().loc[['AAPL']]
clean_data_aapl = clean_data_aapl.reset_index().set_index(['level_1'])

### Save Fundamental Data in SQL

In [63]:
from sqlalchemy import create_engine
import sqlalchemy

# Local Host Postgre SQL Number
sql_post_api = os.getenv("POSTGRES_SQL_API_KEY")

# Define the database
db_name = 'stock_market_price_fundimental_db'

# Define the database URL
db_url = f"{sql_post_api}{db_name}"

# Create the engine object
engine = create_engine(db_url)

# Table name
main_fundamentals_table_name = 'aapl_fundamental_data'
temp_fundamentals = 'temp_aapl_fundamental_data'
temp_main_fundamentals = 'temp_aapl_fundamnetal_db'

# Temparary SQL Table
clean_data_aapl.to_sql(temp_fundamentals, engine, if_exists = 'replace')

# Combine the new API request data with the main SQL database
query = f"""

SELECT * INTO {temp_main_fundamentals} FROM {temp_fundamentals}
UNION
SELECT * FROM {main_fundamentals_table_name};

SELECT *
FROM {temp_main_fundamentals};
"""

# Save and Update SQL database
temp_fundamental_db = pd.read_sql(query, engine)
temp_fundamental_db.to_sql(main_fundamentals_table_name, engine, if_exists = 'replace')

In [90]:
temp_fundamental_db.tail()

,level_1,level_0,Close,Adj. Close,Volume,Piotroski F-score,Sector,Industry,Market Cap,Close Chg,...,Earnings Coverage Ratio Industry,EBIT Yield Industry,EBIT EV Yield Industry,EBITDA Yield Industry,EBITDA EV Yield Industry,Revenue Yield Industry,Revenue EV Yield Industry,Operating Yield Industry,Operating EV Yield Industry,Cash Yield Industry
3345,2020-04-20,AAPL,69.23,68.70,32503750.0,6.0,Technology,Computer Hardware,1.265317e+12,-0.020792,...,1.861081,5.451792,5.033496,7.415575,6.846604,44.265124,40.868819,6.899867,6.370465,9.611299
3346,2020-04-21,AAPL,67.09,66.58,45247893.0,6.0,Technology,Computer Hardware,1.226204e+12,-0.030911,...,1.861081,5.639101,5.192744,7.670353,7.063215,45.785948,42.161813,7.136927,6.572012,9.941516
3347,2020-04-22,AAPL,69.03,68.50,29264342.0,6.0,Technology,Computer Hardware,1.261662e+12,0.028916,...,1.861101,5.475645,5.053864,7.448017,6.874307,44.458676,41.034089,6.930610,6.396755,9.653438
3348,2020-04-23,AAPL,68.76,68.23,31203582.0,6.0,Technology,Computer Hardware,1.256727e+12,-0.003911,...,1.861101,5.486392,5.063018,7.462635,6.886758,44.545935,41.108411,6.944212,6.408341,9.672385
3349,2020-04-24,AAPL,70.74,70.20,31627183.0,6.0,Technology,Computer Hardware,1.292915e+12,0.028796,...,1.860215,5.334682,4.933260,7.256769,6.710714,43.337829,40.076759,6.759268,6.250649,9.500935


## Natural Language Proccessing Database update and IBM Watson Tone Analysis

In [3]:
# Get articles for selected days
start_date = datetime.date(2017, 12, 2)
end_date = datetime.date(2018, 3, 1)
delta = datetime.timedelta(days=1)
articles=[]
while start_date <= end_date:
    gnews_api = os.getenv("gnews_api")
    gnews_url =f"https://gnews.io/api/v4/search?q=apple&in=finance&from={start_date}T00:01:36Z&to={start_date}T23:59:36Z&lang=en&token={gnews_api}"
    response = requests.get(gnews_url)
    data = response.json()
    articles.append(data)
    start_date += delta
    # pause for api restrictions
    time.sleep(4)

In [4]:
# Create Dataframe from JSON file with publish date, title and description
articles_df = pd.json_normalize(articles, record_path = ['articles'], meta = 'totalArticles')
articles_df['title&description']= articles_df['title'] + " " + articles_df['description']
articles_df = articles_df[['publishedAt', 'title&description', 'totalArticles']]
articles_df.rename(columns = {'publishedAt':'date'}, inplace = True)

In [5]:
# Clean Dataframe and group publish date
articles_df['date'] = pd.to_datetime(articles_df['date'], infer_datetime_format=True).dt.date
aapl_articles = articles_df.groupby(by = ["date",'totalArticles']).sum()
aapl_articles = aapl_articles.copy().reset_index().set_index(['date'])

In [6]:
aapl_articles.head()

,totalArticles,title&description
date,,
2018-03-13,1,Chinese Report Expects 11-Inch iPad Pro at WWD...
2018-03-16,1,Steve Jobs Employment Questionnaire Sold for $...
2018-03-18,1,Android P feature spotlight: Apps built for An...
2018-03-23,1,New Low-Cost 9.7-Inch iPad May Support Apple P...
2018-03-24,1,Apple CEO Tim Cook Calls for Stronger Privacy ...


### IBM Watson Tone Analysis of AAPL articles

In [7]:
# Import IBM Watson
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# IBM Watson API Key
ibm_watson_api_key = os.getenv("IBM_WATSON_API_KEY")
authenticator = IAMAuthenticator(ibm_watson_api_key)
tone_analyzer = ToneAnalyzerV3(
    version ='2017-09-21',
    authenticator=authenticator
)

# UBM Watson URL
tone_analyzer.set_service_url('https://api.us-south.tone-analyzer.watson.cloud.ibm.com')
tone_analyzer.set_disable_ssl_verification(True)

In [8]:
# Analyzing the tones in each article
tone_analyzed = []
for text in aapl_articles["title&description"]:
    tone_analysis = tone_analyzer.tone(
    {'text': text},
    content_type ='application/json').get_result()
    tone_analyzed.append(tone_analysis)

C:\Users\palex\.conda\envs\pyvizenv\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.tone-analyzer.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\palex\.conda\envs\pyvizenv\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.tone-analyzer.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\palex\.conda\envs\pyvizenv\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.tone-analyzer.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https

In [9]:
# Converting tones into a dataframe
tone = []
for text in tone_analyzed:
    dic = {}
    tone.append(dic)
    for emotions in text["document_tone"]["tones"]:
        dic.update({emotions['tone_id']:emotions['score']})
aapl_tone_analysis = pd.DataFrame(tone)
aapl_tone_analysis.fillna(0, inplace = True)

# Combining the tone analysis dataframe with the article dataframe
aapl_articles.reset_index(inplace = True)
aapl_nlp = aapl_articles.join(aapl_tone_analysis)
aapl_nlp.set_index("date", inplace = True)

In [10]:
aapl_nlp.head()

,totalArticles,title&description,analytical,tentative,joy,sadness
date,,,,,,
2018-03-13,1,Chinese Report Expects 11-Inch iPad Pro at WWD...,0.881595,0.000000,0.000000,0.0
2018-03-16,1,Steve Jobs Employment Questionnaire Sold for $...,0.000000,0.000000,0.000000,0.0
2018-03-18,1,Android P feature spotlight: Apps built for An...,0.000000,0.672810,0.000000,0.0
2018-03-23,1,New Low-Cost 9.7-Inch iPad May Support Apple P...,0.671030,0.599484,0.000000,0.0
2018-03-24,1,Apple CEO Tim Cook Calls for Stronger Privacy ...,0.000000,0.000000,0.558892,0.0


### Save Data in SQL

In [16]:
from sqlalchemy import create_engine
import sqlalchemy

# Local Host Postgre SQL Number
sql_post_api = os.getenv("POSTGRES_SQL_API_KEY")

# Define the database
db_name = 'stock_market_price_fundimental_db'

# Define the database URL
db_url = f"{sql_post_api}{db_name}"

# Create the engine object
engine = create_engine(db_url)

# Table name
main_nlp_table_name = 'aapl_nlp'
temp_nlp = 'temp_aapl_nlp_data'
temp_main_nlp = 'temp_aapl_nlp_db'

# Temparary SQL Table
aapl_nlp.to_sql(temp_nlp, engine, if_exists = 'replace')

# Combine the new API request data with the main SQL database
query = f"""

SELECT * INTO {temp_main_nlp} FROM {temp_nlp}
UNION
SELECT * FROM {main_nlp_table_name};

SELECT *
FROM {temp_main_nlp};
"""

# Save and Update SQL database
temp_nlp_db = pd.read_sql(query, engine)
temp_nlp_db.to_sql(main_nlp_table_name, engine, if_exists = 'replace')